In [2]:
import sys
sys.path.append('../')
from  mongo.lib.mongo_db  import db_conn
import pandas as pd
import numpy as np
from pyecharts import Bar,configure,Pie
import time
from es_select import ES
import itertools



In [3]:
def date_to_time(date):
    timeArray = time.strptime(date, "%Y%m%d")
    time_semp = time.mktime(timeArray)
    return time_semp   # 2013
def date_to_date(date):
    timeArray = time.strptime(date, "%a %b %d %H:%M:%S %Y")
    time_semp = time.strftime("%Y%m%d%H%M%S", timeArray);
    return time_semp   # 2013

def time_to_today(timeStamp):
    timeArray = time.localtime(int(timeStamp))
    timeArray =time.strftime("%Y-%m-%d %H:%M:%S",timeArray)
    return timeArray   # 2013

## 1、查询20181107日联通出现 5017 （尊敬的客户您好，系统检测您的访问过于频繁，为保障您的账户安全请明天再试。）

In [4]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181107'),'$lte':date_to_time('20181108')},'status':5017})
data1=list(data)

In [5]:
data1_rpt_pd=pd.DataFrame(x for x in data1)

## 2、将代理信息和目标URL取出，统计请求次数、使用代理数量、代理去重列表

In [7]:
def get_proxy(data1_rpt_pd):
    all_sid_http_proxy=[]
    for sid in data1_rpt_pd.sid.values.tolist():
        es=ES('20181108')
        one_sid_http_proxy={}
        #print (sid)
        ret_list=es.get_all_ret(sid)
        #print(len(ret_list))
        one_sid_http_proxy['sid']=sid
        one_sid_http_proxy['tel']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'tel'].values.tolist()[0]
        one_sid_http_proxy['status']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'status'].values.tolist()[0]
        one_sid_http_proxy['http_proxy']=[]
        one_sid_http_proxy['url_base']=[]
        one_sid_http_proxy['dates']=[]
        for i,one_http in enumerate(ret_list):
            one_sid_http_proxy['http_proxy'].append(one_http['_source']['MSG']['proxies']['http'].replace('http://','').split('.')[0])
            one_sid_http_proxy['url_base'].append(one_http['_source']['MSG']['url'])
            one_sid_http_proxy['dates'].append(date_to_date(one_http['_source']['DATETIME']))
        l = [(k, len(list(g))) for k, g in itertools.groupby(one_sid_http_proxy['http_proxy'])]
        one_sid_http_proxy['http_req_count']=l
        one_sid_http_proxy['req_count']=len(ret_list)
        one_sid_http_proxy['http_unique']=list(set(one_sid_http_proxy['http_proxy']))
        one_sid_http_proxy['http_unique_count']=len(one_sid_http_proxy['http_unique'])
        all_sid_http_proxy.append(one_sid_http_proxy)
    return all_sid_http_proxy
        

In [8]:
all_sid_http_proxy=get_proxy(data1_rpt_pd)

In [9]:
all_sid_http_proxy_pd=pd.DataFrame(x for x in all_sid_http_proxy) 

In [10]:
all_sid_http_proxy_pd=all_sid_http_proxy_pd.fillna(0)

## 3、将数据按照手机号码组织，并获取单个客户的请求此次，使用的代理数量

In [11]:
def chain_l(l):
    ret_dict={}
    for one in l:
        if one[0] in ret_dict:
            ret_dict[one[0]]+=one[1]
        else:
            ret_dict[one[0]]=one[1]
    return ret_dict

In [12]:
import itertools
def get_new_tel_list(all_sid_http_proxy_pd):
    new_tel_list=[]
    for tel in all_sid_http_proxy_pd.tel.unique().tolist():
        new_tel_dict={}
        tmp=all_sid_http_proxy_pd.loc[all_sid_http_proxy_pd.tel.values==tel,['tel','req_count','http_req_count','http_unique_count','http_unique']]
        tmp=tmp.fillna(0)
        #print(tmp.shape)
        new_tel_dict['tel']=tel
        new_tel_dict['req_count']=tmp.req_count.values.sum()
        #print(tmp.http_req_count.values.tolist())
        l=list(itertools.chain(*tmp.http_req_count.values.tolist()))
        l_dict=chain_l(l)
        new_tel_dict['http_req_count']=tuple(l_dict.values())
        new_tel_dict['http_unique_count']=tmp.http_unique_count.values.sum()
        new_tel_dict['http_unique']= tuple(l_dict)
        new_tel_list.append(new_tel_dict)
    new_tel_pd=pd.DataFrame(x for x in new_tel_list)
    return new_tel_pd

In [13]:
new_tel_pd=get_new_tel_list(all_sid_http_proxy_pd)

## 4、当天只使用同一个代理但是被封的客户

In [14]:
filte=np.logical_and(new_tel_pd.http_unique_count.values==1,True)
tmp_1=new_tel_pd.loc[filte,['tel','req_count','http_unique_count','http_unique']]

In [15]:
tmp_1

,tel,req_count,http_unique_count,http_unique
2,18599155300,256,1,"(squidsz80,)"
4,13142010961,2,1,"(squidsz75,)"
9,13140368999,28,1,"(squidsz97,)"
21,15647000634,41,1,"(squidsz40,)"
25,18585210620,131,1,"(squidsz73,)"
26,15623253579,22,1,"(squidsz43,)"
30,18659300031,140,1,"(squidsz82,)"


### 电话号码不多，全部反查一下

In [16]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181107'),'$lte':date_to_time('20181108')},'tel':{'$in':tmp_1.tel.values.tolist()}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
one_sid_http_proxy=get_proxy(data2)
one_tel_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy) 
one_tel_http_proxy_pd=get_new_tel_list(one_tel_http_proxy_pd)

In [17]:
one_tel_http_proxy_pd.loc[:,['tel','req_count','http_unique_count','http_unique']]

,tel,req_count,http_unique_count,http_unique
0,13140368999,28,1,"(squidsz97,)"
1,13142010961,2,1,"(squidsz75,)"
2,15623253579,22,1,"(squidsz43,)"
3,15647000634,219,4,"(squidsz27, squidsz15, squidsz40, squidsz97)"
4,18585210620,131,1,"(squidsz73,)"
5,18599155300,256,1,"(squidsz80,)"
6,18659300031,140,1,"(squidsz82,)"


### 由此可以看出当天 15647000634 当天也发生了多次请求，13140368999 发生了一次请求，7日没有成功，8日成功，可能用户在其他渠道操作过，13142010961原因不明

### 接下来分析 代理squidsz75，在这次请求前后是否正常

In [18]:
es=ES('20181108')
proxy_ret=es.get_proxy_ret('http://squidsz75.dianhua.cn:8080')

In [19]:
one_sid_http_proxy_list=[]
for one in proxy_ret:
    one_sid_http_proxy={}
    one_sid_http_proxy['http_proxy']='http://squidsz75.dianhua.cn:8080'
    one_sid_http_proxy['url_base']=(one['_source']['MSG']['url'])
    one_sid_http_proxy['dates']=(date_to_date(one['_source']['DATETIME']))
    one_sid_http_proxy['sid']=((one['_source']['MSG']['sid']))
    one_sid_http_proxy_list.append(one_sid_http_proxy)


In [20]:
def sid_get_err_code(sid):
    data=db_conn.sid_info.find_one({"sid":sid,},{'_id':0,'status':1})
    return data.get('status',1)

In [21]:
one_sid_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy_list)
#one_sid_http_proxy_pd.groupby('sid')['http_proxy'].size().reset_index()
ip_proxy_sort_pd=one_sid_http_proxy_pd.groupby('sid')['dates'].max().reset_index().sort_values(by="dates")
ip_proxy_sort_pd['status']=ip_proxy_sort_pd.sid.map(sid_get_err_code)

In [22]:
ip_proxy_sort_pd

,sid,dates,status
58,SIDd682cfe019b14165bdd5948a23836b51,20181107001621,0
31,SID4da7ba9fb2734cde93eedaedaaade412,20181107002818,0
24,SID3f60c901bcfe4d9fa0a92581e3cc1670,20181107041523,5017
10,SID16a9f0c080184319b98aa04d895207fb,20181107071637,0
60,SIDdd23743f92e4470c98aab8af4dbb687f,20181107084346,0
26,SID42807b0933d64254b3b267156c754055,20181107091936,0
55,SIDcd66979cc85d4e14b581e0b82809c36c,20181107092354,5017
25,SID40f23557b5724db88e7bea9db28804f1,20181107093852,0
51,SIDba682286991649dea22bd282f0e48989,20181107095033,0
27,SID43486c4dd4d94d99943926dc5a585e91,20181107100350,0


#### 13142010961 当天发生的请求为 SIDafe78b5d37aa4339b03b4c1ca69e4743 ，使用代理squidsz75，请求发送在晚上20:31点，出现了请求频繁，该代理之后的其他会话中还是可以正常使用，并爬取成功，怀疑可能该客户当天通过其他渠道操作过账户。

## 5、使用多个代理被封情况分析
req_count 为请求官网次数

http_req_count 为http_unique中代理对应请求次数

http_unique_count 为使用代理个数

http_unique 为被使用过的代理

### A、使用代理数量超过1个的客户

In [23]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,True)
tmp_2=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_2

,tel,req_count,http_req_count,http_unique_count,http_unique
0,13288388003,25,"(9, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1)",11,"(squidsz26, squidsz45, squidsz81, squidsz78, s..."
1,18666151238,240,"(15, 15, 210)",3,"(squidsz01, squidsz118, squidsz48)"
3,18672485919,195,"(19, 15, 4, 13, 15, 129)",7,"(squidsz114, squidsz30, squidsz01, squidsz17, ..."
5,13250512299,4,"(2, 2)",2,"(squidsz73, squidsz72)"
6,18685794878,195,"(15, 30, 15, 4, 2, 129)",7,"(squidsz98, squidsz40, squidsz23, squidsz34, s..."
7,15607201880,278,"(2, 15, 15, 15, 22, 209)",6,"(squidsz21, squidsz15, squidsz01, squidsz75, s..."
8,13015415959,240,"(239, 1)",2,"(squidsz48, squidsz125)"
10,15532356119,6,"(5, 1)",2,"(squidsz84, squidsz85)"
11,18585793920,175,"(15, 15, 15, 130)",4,"(squidsz47, squidsz72, squidsz34, squidsz28)"
12,18681019079,268,"(207, 7, 2, 52)",4,"(squidsz26, squidsz20, squidsz25, squidsz33)"


### B、分析代理总调用次数小于100次的客户

In [24]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values<100)
tmp_3=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_3

,tel,req_count,http_req_count,http_unique_count,http_unique
0,13288388003,25,"(9, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1)",11,"(squidsz26, squidsz45, squidsz81, squidsz78, s..."
5,13250512299,4,"(2, 2)",2,"(squidsz73, squidsz72)"
10,15532356119,6,"(5, 1)",2,"(squidsz84, squidsz85)"
13,13284253781,10,"(5, 1, 1, 1, 1, 1)",6,"(squidsz46, squidsz92, squidsz90, squidsz75, s..."
14,18690566658,14,"(6, 1, 1, 1, 1, 1, 1, 1, 1)",9,"(squidsz76, squidsz83, squidsz85, squidsz111, ..."
15,13049668096,31,"(7, 2, 2, 6, 4, 2, 2, 2, 1, 1, 1, 1)",12,"(squidsz90, squidsz118, squidsz105, squidsz01,..."
16,18695023888,12,"(5, 3, 1, 1, 1, 1)",6,"(squidsz82, squidsz96, squidsz102, squidsz105,..."
18,13138610588,76,"(2, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",36,"(squidsz07, squidsz14, squidsz09, squidsz23, s..."
19,18138764589,10,"(5, 5)",2,"(squidsz130, squidsz26)"
22,15585972487,52,"(7, 45)",2,"(squidsz07, squidsz30)"


#### 查看这些客户当日是否有其他返回码的请求

In [27]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181107'),'$lte':date_to_time('20181108')},'tel':{'$in':tmp_3.tel.values.tolist()}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
some_sid_http_proxy=get_proxy(data2)
some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
some_sid_http_proxy_pd=get_new_tel_list(some_sid_http_proxy_pd)

In [28]:
some_sid_http_proxy_pd

,http_req_count,http_unique,http_unique_count,req_count,tel
0,"(7, 2, 2, 6, 4, 2, 2, 2, 1, 1, 1, 1)","(squidsz90, squidsz118, squidsz105, squidsz01,...",12,31,13049668096
1,"(2, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","(squidsz07, squidsz14, squidsz09, squidsz23, s...",36,76,13138610588
2,"(6, 1, 1, 1, 1, 1)","(squidsz79, squidsz83, squidsz85, squidsz89, s...",6,11,13155152291
3,"(2, 2)","(squidsz73, squidsz72)",2,4,13250512299
4,"(5, 1, 1, 1, 1, 1)","(squidsz46, squidsz74, squidsz75, squidsz50, s...",6,10,13284253781
5,"(9, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1)","(squidsz26, squidsz45, squidsz81, squidsz78, s...",11,25,13288388003
6,"(1, 1, 1)","(squidsz127, squidsz128, squidsz01)",3,3,15518697667
7,"(5, 1)","(squidsz84, squidsz85)",2,6,15532356119
8,"(3, 2, 4, 2, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(squidsz42, squidsz43, squidsz44, squidsz46, s...",19,31,15566370820
9,"(7, 45)","(squidsz07, squidsz30)",2,52,15585972487


#### 这些用户当日不存在成功状态请求，由此推测可能是频繁变更IP地址引起封禁

### C、分析请求数量超过100的代理

In [29]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values>100)
tmp_4=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_4

,tel,req_count,http_req_count,http_unique_count,http_unique
1,18666151238,240,"(15, 15, 210)",3,"(squidsz01, squidsz118, squidsz48)"
3,18672485919,195,"(19, 15, 4, 13, 15, 129)",7,"(squidsz114, squidsz30, squidsz01, squidsz17, ..."
6,18685794878,195,"(15, 30, 15, 4, 2, 129)",7,"(squidsz98, squidsz40, squidsz23, squidsz34, s..."
7,15607201880,278,"(2, 15, 15, 15, 22, 209)",6,"(squidsz21, squidsz15, squidsz01, squidsz75, s..."
8,13015415959,240,"(239, 1)",2,"(squidsz48, squidsz125)"
11,18585793920,175,"(15, 15, 15, 130)",4,"(squidsz47, squidsz72, squidsz34, squidsz28)"
12,18681019079,268,"(207, 7, 2, 52)",4,"(squidsz26, squidsz20, squidsz25, squidsz33)"
17,18596783160,121,"(2, 3, 116)",3,"(squidsz73, squidsz50, squidsz93)"
20,18605979944,147,"(16, 131)",2,"(squidsz105, squidsz78)"
23,13168775881,151,"(15, 115, 21)",3,"(squidsz124, squidsz84, squidsz105)"


#### 使用代理数量相对B情况要少，但是均存在某一个代理使用超过100的情况，可能是因为当日请求量超过阈值或者Ip地址发送变化引起

### D、分析C中代理使用量超过100的情况发生的时机

In [30]:
def select_tel(tel):
    data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181107'),'$lte':date_to_time('20181108')},'tel':tel})
    data2=list(data)
    data2=pd.DataFrame(x for x in data2)
    some_sid_http_proxy=get_proxy(data2)
    some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
    some_sid_http_proxy_pd=some_sid_http_proxy_pd.loc[some_sid_http_proxy_pd.req_count.values>0,['sid','req_count','tel','status','dates']]
    some_sid_http_proxy_pd['dates']=some_sid_http_proxy_pd.dates.map(max)
    return some_sid_http_proxy_pd

In [31]:
select_tel('18672485919')

,sid,req_count,tel,status,dates
0,SID5772afd8cec049b7a4c5c821a9283f3a,129,18672485919,5017,20181107162711
1,SID8b29a6ec3bf448e797c2740f96cb8a72,15,18672485919,5017,20181107163442
2,SID97dbe4a7add640668f87811834c06370,21,18672485919,5017,20181107164347
3,SID34b0c91dc6364c56a08744fdf72deb09,15,18672485919,5017,20181107190859
4,SID3c9211bc11d949e5acf419b2f64941e6,15,18672485919,5017,20181107205353


In [32]:
select_tel('15607201880')

,sid,req_count,tel,status,dates
0,SID1453e31456054813a3f785ed66e59956,209,15607201880,5017,20181107144510
1,SID1430cd325f7a4ceb93542a5f8f73e524,22,15607201880,5017,20181107144521
3,SID874b97fef13d485db7b2f2e1c6f4249c,15,15607201880,5017,20181107145008
4,SIDb9ae4d5a1fdd430a8f983b953964f685,15,15607201880,5017,20181107161726
5,SID1c065905cf3a43e88cebe270b33eefbd,15,15607201880,5017,20181107161928
6,SID5b0679057ae04c3db63b13d629c2cab7,2,15607201880,5017,20181107175627


#### 由此可以看出，客户均在首次请求时已经触发封禁

## 5、结论
### 1）、单次爬虫事务中，如果短时间内请求官网超过131次，可能被封禁
### 2）、多次爬虫事务中，如果使用多个ip代理请求官网，超过3个ip将可能被封禁
### 3）、联通反爬策略为，tel+ip进行识别，相同ip不同客户、相互不干扰
### 4）、针对分析结果，建立爬虫代理池对该以上反爬策略无效